In [486]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from category_encoders import BinaryEncoder
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from scipy.stats import pearsonr, spearmanr, chi2_contingency
from sklearn.metrics import roc_auc_score
from scipy.stats import zscore

from sklearn.decomposition import PCA

from sklearn.metrics import classification_report

In [487]:
# Примерный пайплайн:
# 1) Считываем данные
# 2) Убираем null и лишние поля
# 3) Мерджим датафреймы
# 4) Кодирование категориальных признаков
# 5) опционально - посмотреть корреляцию, убрать лишние признаки
# 6) Масштабирование
# 7) Балансирование классов SMOTE
# 8) Кfold 
# 9) Обучение модели
# 10) Валидация результатов

# сначала базово попробовать настроить нормальный ROC-AUC

# + PCA, регуляризация(???), генерация признаков и PSI

In [488]:
ar = pd.read_csv('data/application_record.zip', compression='zip')
cr = pd.read_csv('data/credit_record.zip', compression='zip')

In [489]:
# общая информация о датасете
# ar.info()

In [490]:
# количество уникальных ID (при общем количестве 438 557)
# ar['ID'].nunique()

In [491]:
# избавляемся от дубликатов в исходном датафрейме
# ar.drop_duplicates('ID', keep='last', inplace=True)
# ar.shape

In [492]:
# общая информация о датасете, содержащем данные о просрочках по кредиту
# cr.info()

In [493]:
# количество уникальных ID
# cr['ID'].unique().size

In [494]:
# смотрим по каким признакам есть нулевые значения
# ar.isnull().sum()

In [495]:
# проверяем наличие нулевых значений с помощью визуализации
# sns.heatmap(ar.isnull())

In [496]:
# слишком много пропущенных значений в поле OCCUPATION_TYPE, избавляемся от него
# ar.drop('OCCUPATION_TYPE', axis=1, inplace=True)


In [497]:
# проверяем наличие нулевых значений с помощью визуализации
# sns.heatmap(ar.isnull())

In [498]:
# # проверяем наличие нулевых значений с помощью визуализации
# sns.heatmap(cr.isnull())

In [499]:
# variances = ar.select_dtypes(include=['number']).var()
# print("Дисперсия признаков:\n", variances)

In [500]:
# очень низкая дисперсия, избавляемся от него
# ar.drop('FLAG_MOBIL', axis=1, inplace=True)

In [501]:
cr.drop('MONTHS_BALANCE', axis=1, inplace=True)

In [502]:
cr['STATUS'].value_counts() 

C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: STATUS, dtype: int64

- **0:** 1-29 days past due 
- **1:** 30-59 days past due 
- **2:** 60-89 days overdue 
- **3:** 90-119 days overdue 
- **4:** 120-149 days overdue 
- **5:** Overdue or bad debts, write-offs for more than 150 days 
- **C:** paid off that month 
- **X:** No loan for the month

In [503]:
cr['STATUS'] = cr['STATUS'].replace({'C' : 0, 'X' : 0})
cr['STATUS'] = cr['STATUS'].astype(int)
cr['STATUS'] = cr['STATUS'].apply(lambda x: 1 if x >= 1 else 0)
cr['STATUS'].value_counts() 

0    1034381
1      14194
Name: STATUS, dtype: int64

In [504]:
df = ar.join(cr.set_index('ID'), on='ID', how='inner')
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0


In [505]:
df.dropna(axis=0, inplace=True)

In [506]:
# # убираем поле с информацией о дате просрочки, оставляем только таргет
# cr = cr.groupby('ID').agg(max).reset_index()
# cr.drop('MONTHS_BALANCE', axis=1, inplace=True)
# cr

In [507]:
# # объединяем в итоговом датафрейме признаки с таргетом
# df = ar.join(cr.set_index('ID'), on='ID', how='inner')
# df.head()

In [508]:
# Вставляем доп.признак с количеством дней просрочки на предпоследнюю позицию
# df.insert(len(df.columns) - 1, 'OVERDUE', df['STATUS'])

# df['OVERDUE'].value_counts()

In [509]:
# df['STATUS'].value_counts() 

In [510]:
# df['STATUS'] = df['STATUS'].apply(lambda x: 1 if x >= 1 else 0)
# df['STATUS'].value_counts() 

In [511]:
# Вычисление матрицы корреляций
# corr_matrix = df.corr()

# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
# plt.title('Матрица корреляций (Пирсон)')
# plt.show()

In [512]:
# correlation_results = []

# # Проходим по всем числовым признакам
# for col in df.select_dtypes(include=['float64', 'int64']).columns:
#     if col != 'target':
#         pearson_corr, _ = pearsonr(df[col], df['STATUS'])
#         spearman_corr, _ = spearmanr(df[col], df['STATUS'])
#         correlation_results.append((col, pearson_corr, spearman_corr))

# # Создаем DataFrame с результатами
# correlation_df = pd.DataFrame(correlation_results, columns=['Feature', 'Pearson', 'Spearman'])
# print(correlation_df.sort_values(by='Pearson', ascending=False))

In [513]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# vif_data = pd.DataFrame()
# vif_data['Feature'] = df.columns
# vif_data['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
# print(vif_data)
# VIF > 5 или 10 указывает на сильную мультиколлинеарность.

In [514]:
# df.drop('CNT_FAM_MEMBERS', axis=1, inplace=True)
# df.drop('CNT_CHILDREN', axis=1, inplace=True) #
# df.drop('NAME_FAMILY_STATUS', axis=1, inplace=True) #
# df.drop('NAME_EDUCATION_TYPE', axis=1, inplace=True) #
# df.drop('DAYS_EMPLOYED', axis=1, inplace=True) #
# df.drop('CODE_GENDER', axis=1, inplace=True)
# df.drop('FLAG_PHONE', axis=1, inplace=True)
# df.drop('FLAG_WORK_PHONE', axis=1, inplace=True)
# df.drop('FLAG_EMAIL', axis=1, inplace=True)

В соответствии с типом признака (номинальный или порядковым) и количеством уникальных значений определим каким образом будем кодировать категориальные переменные.

In [515]:
# смотрим типы данных
# df.dtypes

In [516]:
# object_type = pd.DataFrame(df.dtypes =='object').reset_index()
# object_type = object_type[object_type[0] == True]['index']
# object_type

In [517]:
# df['FLAG_OWN_CAR'].value_counts() # binary encoder

In [518]:
# df['FLAG_OWN_REALTY'].value_counts()

In [519]:
# df['NAME_INCOME_TYPE'].value_counts() # не все значения одинаково часто встречаются, TargetEncoding не подойдет, но порядковый - LabelEncoding

In [520]:
# df['NAME_HOUSING_TYPE'].value_counts() # не все значения одинаково часто встречаются, target encoding не подойдет -> BinaryEncoding

In [521]:
# попробуем просто заменить на 0 и 1
df['FLAG_OWN_CAR'] = df['FLAG_OWN_CAR'].replace({'Y' : 1, 'N' : 0})
df['FLAG_OWN_REALTY'] = df['FLAG_OWN_REALTY'].replace({'Y' : 1, 'N' : 0})
df['CODE_GENDER'] = df['CODE_GENDER'].replace({'F' : 1, 'M' : 0})

In [522]:
#  кодируем категориальный порядковый признак
le = LabelEncoder()
df['NAME_INCOME_TYPE'] = le.fit_transform(df['NAME_INCOME_TYPE'])
df['NAME_HOUSING_TYPE'] = le.fit_transform(df['NAME_HOUSING_TYPE'])
df['NAME_EDUCATION_TYPE'] = le.fit_transform(df['NAME_EDUCATION_TYPE']) #
df['NAME_FAMILY_STATUS'] = le.fit_transform(df['NAME_FAMILY_STATUS']) #
df['OCCUPATION_TYPE'] = le.fit_transform(df['OCCUPATION_TYPE']) #

In [523]:
# mean_encoding = df.groupby('NAME_FAMILY_STATUS')['STATUS'].mean()
# df['NAME_FAMILY_STATUS'] = df['NAME_FAMILY_STATUS'].map(mean_encoding)

In [524]:
# смотрим типы данных
# df.dtypes

In [525]:
# df

In [526]:
# df['STATUS'].value_counts()

In [527]:
# # Вычисление матрицы корреляций
# corr_matrix = df.corr()

# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
# plt.title('Матрица корреляций (Пирсон)')
# plt.show()

In [528]:
# correlation_results = []

# # Проходим по всем числовым признакам
# for col in df.select_dtypes(include=['float64', 'int64']).columns:
#     if col != 'target':
#         pearson_corr, _ = pearsonr(df[col], df['STATUS'])
#         spearman_corr, _ = spearmanr(df[col], df['STATUS'])
#         correlation_results.append((col, pearson_corr, spearman_corr))

# # Создаем DataFrame с результатами
# correlation_df = pd.DataFrame(correlation_results, columns=['Feature', 'Pearson', 'Spearman'])
# print(correlation_df.sort_values(by='Pearson', ascending=False))

In [529]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# vif_data = pd.DataFrame()
# vif_data['Feature'] = df.columns
# vif_data['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
# print(vif_data)
# VIF > 5 или 10 указывает на сильную мультиколлинеарность.

In [530]:
# plt.figure(figsize=(8, 6))
# sns.boxplot(x='STATUS', y='AMT_INCOME_TOTAL', data=df)
# plt.title('Распределение дохода для дефолтных и недефолтных заемщиков')
# plt.show()

In [531]:
# # проверяем наличие выбросов с помощью визуализации

# fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(18,10))
# sns.scatterplot(x=df['ID'], y=df['FLAG_OWN_CAR'], data=df, ax=ax[0][0])
# sns.scatterplot(x=df['ID'], y=df['FLAG_OWN_REALTY'], data=df, ax=ax[0][1])
# sns.scatterplot(x=df['ID'], y=df['AMT_INCOME_TOTAL'], data=df, ax=ax[0][2])

# sns.scatterplot(x=df['ID'], y=df['NAME_INCOME_TYPE'], data=df, ax=ax[1][0])
# sns.scatterplot(x=df['ID'], y=df['NAME_HOUSING_TYPE'], data=df, ax=ax[1][1])
# sns.scatterplot(x=df['ID'], y=df['FLAG_WORK_PHONE'], data=df, ax=ax[1][2])

# sns.scatterplot(x=df['ID'], y=df['FLAG_PHONE'], data=df, ax=ax[2][0])
# sns.scatterplot(x=df['ID'], y=df['FLAG_EMAIL'], data=df, ax=ax[2][1])

In [532]:
# # Вычисляем Z-оценки и удаляем выбросы (|Z| > 3)
# df['z_score'] = zscore(df['AMT_INCOME_TOTAL'])
# df = df[np.abs(df['z_score']) <= 3]

In [533]:
# # Вычисляем Z-оценки и удаляем выбросы (|Z| > 3)
# df['z_score'] = zscore(df['NAME_INCOME_TYPE'])
# df = df[np.abs(df['z_score']) <= 3]

In [534]:
# # проверяем наличие выбросов с помощью визуализации после очистки

# fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(18,10))
# sns.scatterplot(x=df['ID'], y=df['FLAG_OWN_CAR'], data=df, ax=ax[0][0])
# sns.scatterplot(x=df['ID'], y=df['FLAG_OWN_REALTY'], data=df, ax=ax[0][1])
# sns.scatterplot(x=df['ID'], y=df['AMT_INCOME_TOTAL'], data=df, ax=ax[0][2])

# sns.scatterplot(x=df['ID'], y=df['NAME_INCOME_TYPE'], data=df, ax=ax[1][0])
# sns.scatterplot(x=df['ID'], y=df['NAME_HOUSING_TYPE'], data=df, ax=ax[1][1])
# sns.scatterplot(x=df['ID'], y=df['FLAG_WORK_PHONE'], data=df, ax=ax[1][2])

# sns.scatterplot(x=df['ID'], y=df['FLAG_PHONE'], data=df, ax=ax[2][0])
# sns.scatterplot(x=df['ID'], y=df['FLAG_EMAIL'], data=df, ax=ax[2][1])

In [535]:
# генерация признаков + PSI 

In [536]:
# df['DAYS_BIRTH'] = - (df['DAYS_BIRTH'] // 365)
# df['DAYS_BIRTH']

In [537]:
# plt.hist(df['DAYS_BIRTH'], bins=10, edgecolor='black')
# plt.show()

In [538]:
# # Определяем границы бинов
# bins = [20, 30, 40, 50, 60, 100]

# # Создаем метки для бинов
# labels = ['21-30', '31-40', '41-50', '51-60', '60+']

# # Разбиваем возраст на бины
# df['DAYS_BIRTH'] = pd.cut(df['DAYS_BIRTH'], bins=bins, labels=labels, right=False)

In [539]:
# df['DAYS_BIRTH'].value_counts()

In [540]:
# mean_encoding = df.groupby('DAYS_BIRTH')['STATUS'].mean()
# df['DAYS_BIRTH'] = df['DAYS_BIRTH'].map(mean_encoding)

In [541]:
# encoder = BinaryEncoder(cols=['DAYS_BIRTH'])
# df = encoder.fit_transform(df)

In [542]:
df

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
2,5008806,0,1,1,0,112500.0,4,4,1,1,-21474,-1134,1,0,0,0,16,2.0,0
2,5008806,0,1,1,0,112500.0,4,4,1,1,-21474,-1134,1,0,0,0,16,2.0,0
2,5008806,0,1,1,0,112500.0,4,4,1,1,-21474,-1134,1,0,0,0,16,2.0,0
2,5008806,0,1,1,0,112500.0,4,4,1,1,-21474,-1134,1,0,0,0,16,2.0,0
2,5008806,0,1,1,0,112500.0,4,4,1,1,-21474,-1134,1,0,0,0,16,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434812,5150337,0,0,1,0,112500.0,4,4,3,4,-9188,-1193,1,0,0,0,8,1.0,0
434812,5150337,0,0,1,0,112500.0,4,4,3,4,-9188,-1193,1,0,0,0,8,1.0,1
434812,5150337,0,0,1,0,112500.0,4,4,3,4,-9188,-1193,1,0,0,0,8,1.0,1
434812,5150337,0,0,1,0,112500.0,4,4,3,4,-9188,-1193,1,0,0,0,8,1.0,0


In [543]:
df['STATUS'].value_counts() 

0    529282
1      8385
Name: STATUS, dtype: int64

In [544]:
# df

In [545]:
X = df.iloc[:,1:-1]  # с конца убираем таргет 
y = df['STATUS']

In [546]:
# X_PCA = PCA(n_components=3).fit_transform(X)

In [547]:
# делим данные на обучающую и тестовую выборки
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [548]:
# приводим признаки к одной шкале, RobustScaler не чувствителен к выбросам в отличие от MinMaxScaler()
# rs = RobustScaler()
# rs = MinMaxScaler()
# X_scaled = rs.fit_transform(X_train)
# X_test_scaled = rs.transform(X_test)
# X_scaled = pd.DataFrame(rs.fit_transform(X_train), columns=X_train.columns)
# X_test_scaled = pd.DataFrame(rs.transform(X_test), columns=X_test.columns)

In [549]:
# # приводим признаки к одной шкале, MinMaxScaler() чувствителен к выбросам 
# rs = MinMaxScaler()
# X_scaled = pd.DataFrame(rs.fit_transform(X_train), columns=X_train.columns)
# X_test_scaled = pd.DataFrame(rs.transform(X_test), columns=X_test.columns)

In [550]:
# my version

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

rs = MinMaxScaler()
X_scaled = rs.fit_transform(X_train)
X_test_scaled = rs.transform(X_test)

smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_scaled, y_train)

xgb = XGBClassifier(random_state = 42, n_estimators = 300)
model = xgb.fit(X_balanced, y_balanced)
xgb_prediction = xgb.predict(X_test_scaled)

train_score = xgb.score(X_balanced, y_balanced)
test_score = xgb.score(X_test, y_test)

xgb train score = 0.9126463304959429
xgb test score = 0.984406734267243
              precision    recall  f1-score   support

           0       0.99      0.89      0.94    132321
           1       0.09      0.68      0.16      2096

    accuracy                           0.88    134417
   macro avg       0.54      0.79      0.55    134417
weighted avg       0.98      0.88      0.93    134417

xgb AUC: 0.7857960773999108
xgb Gini: 0.5715921547998215


In [ ]:
print(f"xgb train score = {train_score}")
print(f"xgb test score = {test_score}")

print(classification_report(y_test, xgb_prediction)) #1 xgb

# Вычисляем AUC
auc = roc_auc_score(y_test, xgb_prediction)

# Вычисляем коэффициент Джини
gini = 2 * auc - 1
print(f"xgb AUC: {auc}")
print(f"xgb Gini: {gini}")


cm = confusion_matrix(y_test, xgb_prediction)
print("Confusion Matrix:\n", cm)

In [555]:
# def calculate_psi(expected, actual, bins=10, epsilon=1e-6):
#     # Разбиваем данные на бины
#     breakpoints = np.histogram_bin_edges(expected, bins=bins)
#     expected_counts, _ = np.histogram(expected, bins=breakpoints)
#     actual_counts, _ = np.histogram(actual, bins=breakpoints)
    
#     # Добавляем epsilon, чтобы избежать деления на ноль
#     expected_counts = expected_counts + epsilon
#     actual_counts = actual_counts + epsilon
    
#     # Нормализуем до процентов
#     expected_percents = expected_counts / np.sum(expected_counts)
#     actual_percents = actual_counts / np.sum(actual_counts)
    
#     # Рассчитываем PSI
#     psi = np.sum((actual_percents - expected_percents) * np.log(actual_percents / expected_percents))
    
#     return psi

# def calculate_psi_for_dataframe(df_expected, df_actual, bins=10):
#     psi_results = {}
    
#     for column in df_expected.columns:
#         expected = df_expected[column]
#         actual = df_actual[column]
#         psi_value = calculate_psi(expected, actual, bins=bins)
#         psi_results[column] = psi_value
    
#     return psi_results

In [556]:
# # Рассчитываем PSI для всех признаков
# psi_results = calculate_psi_for_dataframe(X_balanced, X_test_balanced, bins=10)

# # Выводим результаты
# for feature, psi in psi_results.items():
#     print(f"PSI для признака {feature}: {psi:.4f}")

In [557]:
# classifiers = {
#     "LogisticRegression" : LogisticRegression(max_iter=1000),
#     "KNeighbors" : KNeighborsClassifier(),
#     "SVC" : SVC(),
#     "DecisionTree" : DecisionTreeClassifier(),
#     "RandomForest" : RandomForestClassifier(),
#     "XGBoost" : XGBClassifier()
# }

In [558]:
# train_scores = []
# test_scores = []

# for key, classifier in classifiers.items():
#     classifier.fit(X_balanced, y_balanced)
#     train_score = classifier.score(X_balanced, y_balanced)
#     train_scores.append(train_score)
#     test_score = classifier.score(X_test, y_test)
#     test_scores.append(test_score)

# print(train_scores)
# print(test_scores)

In [563]:
# import shap

# # Создание explainer
# explainer = shap.TreeExplainer(model)
# shap_values = explainer.shap_values(X_balanced)

# # Визуализация значимости признаков
# shap.summary_plot(shap_values, X_balanced, feature_names=X_balanced.columns)

In [564]:
# explainer = shap.TreeExplainer(model)
# shap_values = explainer(X_balanced)
# shap.plots.waterfall(shap_values[0])

In [565]:
# explainer = shap.TreeExplainer(model)
# shap_values = explainer(X_balanced)
# shap.plots.waterfall(shap_values[1])

In [566]:
# shap.plots.bar(shap_values)

In [567]:
# logreg = LogisticRegression(max_iter=1000)
# model = logreg.fit(X_balanced, y_balanced)
# train_score = logreg.score(X_balanced, y_balanced)
# test_score = logreg.score(X_test, y_test)
# logreg_prediction = logreg.predict(X_test)

# print(f"logreg train score = {train_score}")
# print(f"logreg test score = {test_score}")

In [568]:
# print(classification_report(y_test, logreg_prediction)) #logreg

In [569]:
# # Вычисляем AUC
# auc = roc_auc_score(y_test, logreg_prediction)

# # Вычисляем коэффициент Джини
# gini = 2 * auc - 1
# print(f"logreg AUC: {auc}")
# print(f"logreg Gini: {gini}")